# Predicting the Geography of Online Sales in Emerging Markets
The objective of this kernel is to create a model that is capable of predicting sales volumes with respect to geolocation. This could potentially be used by future E-Commerce ventures to predict consumer behavior in other emerging markets outside of Brazil, before the business even begins accepting orders.

## Methodology
The Brazilian E-Commberce Public Dataset by Olist will be used to provide the total sales volume at a number of geospatial points. Additional economic and infrastructure data will be brought in from the following sources:
- Airport geospatial data from anac.gov.br
- Airport cargo stastistics from infraero.gov.br
- City geospatial and population data from ???????
- Regional economical and population data from ???????

The listed data sources will be merged to obtain several features describing each geospatial point, and those features will be used to train a model to predict sales volumes at any given geospatial location.

To begin, we look at the Olist geolocation dataset.

In [ ]:
import numpy as np
import pandas as pd 
import os
###Get a geospatial description of the zip codes:
geo_df = pd.read_csv("../input/olist_geolocation_dataset.csv", dtype={'geolocation_zip_code_prefix': str})
geo_df.head(5)

While the geolocation dataset provides us with multiple locations for each zip code, we can only use one location for each zip code when we merge with the other data. The most obvious approach is to take the mean location for each zip code and use that. This will work fine assuming that each zip code spans a small area. We can check this assumption just by using the standard deviation.

In [ ]:
# x, y = webm(geo.geolocation_lng, geo.geolocation_lat)
# geo_df['x'] = pd.Series(x)
# geo_df['y'] = pd.Series(y)
#Look at each unique zip code individually:
unique_zips = np.unique(geo_df['geolocation_zip_code_prefix'].values)
geo_df.loc['geolocation_zip_code_prefix'=='01001']
geo_grp = geo_df.groupby(['geolocation_zip_code_prefix']).agg(['mean','median','std','count'])
geo_grp.sort_values(by=('geolocation_lat','std'), ascending=False).head(5)

...and clearly something is wrong here. Some of these zip codes have astronomically high standard deviations, meaning they contain points sprawled over a huge area. We can look at one example to see why.

In [ ]:
###Look at the points within one of the bad zip codes:
BAD_ZIP = ['28595']
plot_df = geo_df.loc[geo_df['geolocation_zip_code_prefix'].isin(BAD_ZIP)].copy()
plot_df

We can see that we have one major outlier here. And after manually inspecting a few others of these zip codes, I can see that outliers are not so uncommon. 

Since many of these zip codes contain so few points, and we can't guarentee that we only have a single outlier, outlier detection is not always possible. The safest approach is to discard any zip codes that only contain one point, and also discard zip codes with small numbers of points and high standard deviations. Once those zip codes have been filtered out, any outliers in other sets can be neutralized by using the median location, rather than the mean.

In [ ]:
print('Total number of zip codes:                                               {}'.format(geo_grp.shape[0]))
filt_1 = geo_grp['geolocation_lat','count']==1
geo_grp = geo_grp.loc[~filt_1]
print('Number of zip codes with only one point:                                 {}'.format(filt_1.sum()))
filt_2 = (geo_grp['geolocation_lat','count']<5) & ((geo_grp['geolocation_lat','std']>0.1) | (geo_grp['geolocation_lng','std']>0.1))
geo_grp = geo_grp.loc[~filt_2]
print('Number of zip codes with less than 5 points and high standard deviation: {}'.format(filt_2.sum()))
print('Remaining number of zip codes:                                           {}'.format(geo_grp.shape[0]))



We see that even after applying a rather conservative filter to eliminate questionable data, we are still left with 17857 usable zip codes.

## Brazil's airports

The size of and distance to the nearest airport is indicitive of existing economic conditions in an area, and also the cost of shipping to that area. All of these implications will influence local customers' potential spending habits.

To utilize these data, we first need to download and clean the data. Then merge the data. Later, we can compare customers' locations to the airport locations to develop features for our training model.

We begin with the airport cargo statistics data.

In [ ]:
#Download cargo statitics for Brazil's airports
import requests
r = requests.get('http://www4.infraero.gov.br/media/673983/jun.xlsx')
if r.status_code == 200:
    with open('jun.xlsx','wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)
# os.stat('./jun.xlsx')
cargo_df = pd.read_excel('./jun.xlsx', sheet_name=1, header=4, usecols='B,H').dropna()
cargo_df.head(10)

In [ ]:
###Clean up the cargo data
#Replace the column names with english equivalents:
cargo_df.columns = ['Description', 'Cargo loaded and unloaded per year']
#Keep only the main row for each airport:
matches = cargo_df['Description'].str.contains('[A-Z]{4} - ')
cargo_df = cargo_df[matches]
#Separate the airport codes from the common names
cargo_df['ICAO code'], cargo_df['Airport name'] = cargo_df['Description'].str.split(' - ').str
cargo_df = cargo_df.drop(columns=['Description'])
#Reorder the columns:
cargo_df = cargo_df.reindex(columns=['ICAO code','Airport name', 'Cargo loaded and unloaded per year'])
cargo_df.head(5)

Next, we move on to the airport geolocation data.

In [ ]:
!wget http://www2.anac.gov.br/arquivos/pdf/aerodromos/AerodromosPublicos.xls

In [ ]:
import re
###Clean up the airport location data
airport_df = pd.read_excel('./AerodromosPublicos.xls', header=1, usecols='A,F,G')
#Replace the column names with english equivalents:
airport_df.columns = ['ICAO code', 'Latitude_str','Longitude_str']
#Convert lat/lon strings to decimal values:
def latlon_get_decimal(input_str):
    X = re.search("(\d+)(?:°\s*)(\d+)(?:'\s*)(\d+)(?:''\s*)([A-Z])", input_str)
    dval = int(X.group(1)) + int(X.group(2))/60 + int(X.group(3))/3600
    if X.group(4) in ['S','W']:
        dval = dval * -1
    return dval
airport_df['Lat'] = airport_df['Latitude_str'].apply(latlon_get_decimal)
airport_df['Lon'] = airport_df['Longitude_str'].apply(latlon_get_decimal)
airport_df.head(5)

In [ ]:
###Merge cargo data with airport location data:
airportcargo = cargo_df.merge(airport_df, on='ICAO code')
airportcargo.head(10)

## Brazil's States
Brazil's 27 states (officially called federative units) each cover a vast area. With the largest, Amazonas, being nearly half the size of India. While the large areas are not ideal for our purposes, the detailed economic data at this level makes them useful nonetheless.

I found a number of good resources showing various data at the state level, but to try something a little different, we will scrape a table of data from __[this Wikipedia page.](https://en.wikipedia.org/wiki/States_of_Brazil)__

Pandas' `read_html()` method makes this very convenient.

In [ ]:
#Scrape the raw html source from the webpage:
html = requests.get("https://en.wikipedia.org/wiki/States_of_Brazil").text
#Use pandas to parse the table from the html source
state_df = pd.read_html(html,attrs = {'class':"wikitable sortable"}, header=0, flavor='bs4')[0]
#Remove some unwanted columns:
state_df = state_df.drop(['Flag','Capital','Area (sq mi)','Density (per sq mi, 2017)'], axis=1)
#This column contains two values per cell, but we only want the first one:
state_df['GDP (billion R$, 2012)'] = state_df['GDP (billion R$ and\xa0% total, 2012)'].apply(
    lambda x: float(x.split('(')[0].replace(',','')))
state_df = state_df.drop('GDP (billion R$ and\xa0% total, 2012)', axis=1)
#These columns contain percentages. Convert them to rate values:
state_df['Literacy (2014)'] = state_df['Literacy (2014)'].apply(
    lambda x: float(x.split('%')[0])/100)
state_df['Infant mortality (2014)'] = state_df['Infant mortality (2014)'].apply(
    lambda x: float(x.split('%')[0])/100)
state_df.head(5)

Later on, we will also need a way to correlate state abbreviations with their full names. This dataframe gives us a convenient opportunity to create a dictionary object with this functionality.

In [ ]:
state_abbr = dict(np.fliplr(state_df[['Federative unit', 'Abbreviation']].values))
state_abbr

## Brazil's Cities
An area's economy will be dependent on its distance to the nearest major city center, and also the level of economic development within that city. Unfortunately, I was unable to locate publicly available data describing economic develpment across Brazil's cities, so the population of each city will be used to infer economic development. This is obviously non-ideal, but we will try to make do.



In [ ]:
!wget ftp://ftp.ibge.gov.br/Estimativas_de_Populacao/Estimativas_2017/estimativa_TCU_2017_20181108.xls

In [ ]:
city_df = pd.read_excel('./estimativa_TCU_2017_20181108.xls', sheet_name=1, header=1,
                       dtype=str)
city_df.head(20)

This table needs some clean-up as well. In particular, we see some irregularities in numbers of the final column.

In [ ]:
#Replace the column names with english equivalents:
city_df.columns = ['State', 'State code','City code','Municipality','Estimated population']
#Remove irregularities in the population numbers:
def get_population_number(input_str):
    input_str = input_str.split('(')[0]
    input_str = input_str.replace('.','')
    try:
        return int(input_str)
    except ValueError:
        return np.nan
city_df['Estimated population'] = city_df['Estimated population'].apply(get_population_number)
city_df = city_df.dropna()
#We dont really have any use for the state and city codes, so drop them:
city_df = city_df.drop(['State code','City code'], axis=1)
city_df.head(5)

I was unable to find tabulated data showing the geoposition of each city. So instead we will grab that data from Wikipedia using the very convenient wikipedia package.

At first I attempted to search for each city name in sucession, but this method is slow and produces many irregular results (such as in cases where there are several cities sharing the same name). A far better method is to use [this wikipedia page](https://en.wikipedia.org/wiki/List_of_municipalities_of_Brazil), which tabulates the links to the articles on every Brazilian municipality.

In [ ]:
from bs4 import BeautifulSoup
#Scrape the raw html source from the webpage:
html = requests.get("https://en.wikipedia.org/wiki/List_of_municipalities_of_Brazil").text
#read_html() can only get the text from the table:
links_df = pd.read_html(html,attrs = {'class':'wikitable sortable'}, header=0, flavor='bs4')[0]
#Rather than keeping the entire state name, it will be more convenient to only keep the abbreviation:
links_df['State'] = links_df['State'].apply(
    lambda x: x.split('(')[1].split(')')[0])
#Remove the columns that aren't useful to us:
links_df = links_df.drop(['Mesoregion','Microregion'], axis=1)

###Use beautifulsoup to parse hrefs from the table:
b = BeautifulSoup(html,'lxml')
table = b.find(name='table')
table_cells = np.array([td for td in table.find_all(name='td')])
#Take the links from every 4th cell in the table:
muni_links = np.array([td.a['href'] for td in table_cells[3::4]])
#Add the links to our dataframe:
links_df['Link'] = muni_links
links_df.head(5)

In [ ]:
###Merge the city population dataframe with the links:
#Change the capitalization of names in city_df so that they match the wikipedia names:
city_df['Municipality'] = city_df['Municipality'].apply(
    lambda x: x.replace("D'", "d'"))
#Try the merge, but check which cities didn't match up with a wikipedia link:
city_df = city_df.merge(links_df, how='left', on=['State','Municipality'])
unmatched = city_df.loc[pd.isnull(city_df['Link'])]
print('{} cities did not match a wikipedia entry'.format(unmatched.shape[0]))
unmatched.head(5)

In [ ]:
###Use difflib for fuzzy string matching:
import difflib

def remove_accents(input_str):
    input_str = input_str.replace('á', 'a')
    input_str = input_str.replace('é', 'e')
    input_str = input_str.replace('ê', 'e')
    input_str = input_str.replace('í', 'i')
    input_str = input_str.replace('ó', 'o')
    input_str = input_str.replace('ú', 'u')
    return input_str

for idx,row in city_df.loc[pd.isnull(city_df['Link'])].iterrows():
    #Take the portion of the links_df that matches the state
    choices = links_df.loc[links_df['State']==row['State']]
    #difflib seems to struggle to match strings with different accent marks, so remove them:
    city_name = remove_accents(row['Municipality'])
    choices['Municipality'] = choices['Municipality'].apply(remove_accents)
    #Find which choice best matches the unfinished row:
    match = difflib.get_close_matches(city_name, 
                                      choices['Municipality'].values, 
                                      n=1, 
                                      cutoff=0.8)
    if match:
        city_df.loc[idx,'Link'] = choices.loc[choices['Municipality']==match[0],'Link'].values[0]
#Check again and see how many cities are still yet to find a match:
unmatched = city_df.loc[pd.isnull(city_df['Link'])]
print('{} cities did not match a wikipedia entry'.format(unmatched.shape[0]))
unmatched.head(50)

Now we're down to just 18 cities without a match. Not bad considering that we started with over 5000 cities.

The remaining cities can be matched by using the search function in the wikipedia API.

In [ ]:
###Scrape city data from Wikipedia.org
import wikipedia
wikipedia.set_lang("en")

#Search for each city that still hasn't found a match:
for idx,row in city_df.loc[pd.isnull(city_df['Link'])].iterrows():
    search_str = row['Municipality'] + ', ' + state_abbr[row['State']]
    page = wikipedia.page(search_str)
    print(search_str)
    print(page.title)
    print(page.url)

# """
# We need to look at the individual list for each state (eg. /wiki/List_of_municipalities_in_Amap%C3%A1)
#  and then find the corresponding municipality within that webpage. That link is the most reliable way 
#  to arrive at the correct wikipedia article.
# """
# muni = wikipedia.page('Municipalities of Brazil')

In [ ]:
# def get_city_coords(city_name):
#     #Some of the names in city_df appear like "Machadinho D'Oeste", 
#     #  while wikipedia changes the capitialization to "Machadinho d'Oeste"
#     city_name = city_name.replace("D'","d'")
#     try:
#         muni = wikipedia.page(city_name, auto_suggest=False)
#         return float(muni.coordinates[0]), float(muni.coordinates[1])
#     except wikipedia.DisambiguationError:
#         print('---DisambiguationError---')
#         print(city_name)
#     except KeyError:
#         print('---KeyError---')
#         print(city_name)
#     except wikipedia.PageError:
#         print('---PageError---')
#         print(city_name)
#     return np.nan, np.nan
# city_df['Lat','Lon'] = city_df['City name'].apply(get_city_coords)
# city_df